In [2]:
%cd ..

/home/anthony/workspace/yt-data


In [3]:

import tensorflow as tf
import os
from typing import List, Tuple, Iterator
from rangedict import RangeDict
import numpy as np

from thumbs.experiment import Experiment
from thumbs.loss import Loss
from thumbs.data import get_pokemon_data256
from thumbs.params import HyperParams, MutableHyperParams
from thumbs.model.model import Model, BuiltModel

from keras.models import Sequential
from keras.layers import Dense, Reshape, Conv2DTranspose, Flatten, LeakyReLU
from keras.layers import (
    Activation,
    BatchNormalization,
    Dense,
    GaussianNoise,
    Dropout,
    Flatten,
    Reshape,
    ReLU,
    LeakyReLU,
    LayerNormalization,
    # BatchNormalizationV2,
)

# from keras.layers.normalization.batch_normalization_v1 import (
#     BatchNormalization,
# )
from tensorflow.compat.v1.keras.layers import BatchNormalization as BatchNormalizationV1
from keras.layers.convolutional import Conv2D, Conv2DTranspose

from thumbs.train import Train, TrainMSE, TrainBCE, TrainBCESimilarity, TrainWassersteinGP

/home/anthony/workspace/yt-data/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
n_hidden_layers = 6

ngf = 64

def build_generator(z_dim):
    model = Sequential(name="generator")

    model.add(Reshape((1, 1, z_dim), input_shape=(z_dim,)))
    model.add(Conv2DTranspose(ngf*4, kernel_size=6, strides=6, padding='valid'))

    model.add(Conv2DTranspose(ngf*3, kernel_size=5, strides=5, padding='valid'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(ngf*2, kernel_size=5, strides=2, padding='valid'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(ngf, kernel_size=2, strides=1, padding='valid'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(3, kernel_size=2, strides=2, padding="valid"))
    model.add(Activation("tanh"))

    # model.add(Conv2DTranspose(128, kernel_size=2, strides=1))

    # # model.add(Dense(1024* 8 * 8, input_dim=z_dim))
    # # model.add(Reshape((8, 8, 1024)))
    # model.add(Reshape((1, 1, z_dim), input_shape=(z_dim,)))

    # # get 6 numbers from an interpolate between 1 4
    # for f in np.linspace(4, 1, n_hidden_layers):
    #     model.add(Conv2DTranspose(ngf*f, kernel_size=5, strides=2, padding="same"))
    #     if f != 4:
    #         # Don't normalize the first layerlayer
    #         model.add(BatchNormalization())
    #     model.add(ReLU())



    # model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding="valid"))
    # model.add(Activation("tanh"))

    model.summary(200)
    return model

build_generator(100)

Model: "generator"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                                             Output Shape                                                                    Param #                       
 reshape_39 (Reshape)                                                                     (None, 1, 1, 100)                                                               0                             
                                                                                                                                                                                                        
 conv2d_transpose_113 (Conv2DTranspose)                                                   (None, 6, 6, 256)                                                               921856 